# ReAct Agent with LangChain

This notebook demonstrates how to create and use a ReAct (Reasoning and Acting) agent with LangChain.

ReAct agents combine reasoning and acting by allowing the LLM to:
1. **Reason** about what action to take
2. **Act** by calling tools
3. **Observe** the results
4. Repeat until the task is complete

## What You'll Learn:
- Setting up a ReAct agent
- Integrating tools (Tavily Search)
- Running agent workflows
- Understanding agent decision-making process

## 1. Setup and Imports

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Import our custom Tavily search tool
from tools.tavily_search import get_tavily_search_tool

# Load environment variables
load_dotenv()

print("✓ Imports successful!")

## 2. Set Up API Keys

You'll need:
- OpenAI API key (for the LLM)
- Tavily API key (for web search)

Set these as environment variables or in a `.env` file.

In [ ]:
# Uncomment and set your API keys if not using .env file
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["TAVILY_API_KEY"] = "your-tavily-api-key"

# Verify keys are set
if "OPENAI_API_KEY" in os.environ and "TAVILY_API_KEY" in os.environ:
    print("✓ API keys are configured!")
else:
    print("⚠ Warning: Please set your API keys")

## 3. Initialize the LLM

We'll use OpenAI's GPT-4 model as our reasoning engine.

In [ ]:
# Initialize the LLM
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0,  # Lower temperature for more consistent reasoning
)

print(f"✓ LLM initialized: {llm.model_name}")

## 4. Set Up Tools

Tools are the functions/APIs that the agent can use to interact with the world.
We'll use the Tavily search tool to enable web searching.

In [ ]:
# Get the Tavily search tool
search_tool = get_tavily_search_tool(
    max_results=5,
    search_depth="advanced"
)

# Create tools list
tools = [search_tool]

print(f"✓ Tools configured: {[tool.name for tool in tools]}")

## 5. Create the ReAct Agent

The ReAct agent uses a specific prompt template that guides the LLM to:
- Think through problems step-by-step
- Decide which tools to use
- Process observations from tool outputs

In [ ]:
# Get the ReAct prompt template from LangChain Hub
prompt = hub.pull("hwchase17/react")

# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create an agent executor (manages the agent's execution)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # Show the agent's reasoning process
    handle_parsing_errors=True,  # Handle any parsing errors gracefully
    max_iterations=10  # Prevent infinite loops
)

print("✓ ReAct agent created!")

## 6. Example 1: Simple Information Retrieval

Let's ask the agent to find current information.

In [ ]:
# Run the agent with a query
result = agent_executor.invoke({
    "input": "What are the latest developments in artificial intelligence in 2024?"
})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
print(result["output"])

## 7. Example 2: Multi-Step Reasoning

Now let's try a more complex query that requires multiple reasoning steps.

In [ ]:
# Complex query requiring research and synthesis
result = agent_executor.invoke({
    "input": "Who is the current CEO of OpenAI and what is their background?"
})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
print(result["output"])

## 8. Example 3: Comparative Analysis

The agent can perform comparative analysis by gathering multiple pieces of information.

In [ ]:
# Comparative query
result = agent_executor.invoke({
    "input": "Compare the main features of GPT-4 and Claude 3. What are the key differences?"
})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
print(result["output"])

## 9. Understanding the ReAct Loop

The ReAct agent follows this pattern:

```
Thought: I need to find information about X
Action: tavily_search
Action Input: "X latest information"
Observation: [Search results]
Thought: Based on the results, I can answer...
Final Answer: [Agent's response]
```

This loop continues until the agent determines it has enough information to provide a final answer.

## 10. Custom Query

Try your own query!

In [ ]:
# Your custom query here
custom_query = "What is LangChain and how is it used in AI applications?"

result = agent_executor.invoke({"input": custom_query})

print("\n" + "="*80)
print("FINAL ANSWER:")
print("="*80)
print(result["output"])

## Summary

In this notebook, you learned:
- ✓ How to set up a ReAct agent with LangChain
- ✓ How to integrate tools (Tavily search) with agents
- ✓ How the ReAct reasoning loop works
- ✓ How to use agents for information retrieval and analysis

## Next Steps
- Explore adding more tools to the agent
- Try different LLM models
- Experiment with custom prompts
- Check out `01.deep_agents_basic.ipynb` for more advanced agent patterns